In [31]:
import csv
from bs4 import BeautifulSoup

In [32]:
#firefox and chrome
from selenium import webdriver

In [33]:
#microsoft edge
from msedge.selenium_tools import Edge, EdgeOptions

### Startup the webdriver

In [34]:
#firefox and chrome
# driver = webdriver.FireFox()
# driver = webdriver.Chrome()

In [37]:
#chrome
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))




====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
There is no [win32] chromedriver for browser 100.0.4896 in cache
Trying to download new driver from https://chromedriver.storage.googleapis.com/100.0.4896.60/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\CUONG NGUYEN\.wdm\drivers\chromedriver\win32\100.0.4896.60]


In [38]:
url = 'https://www.amazon.com/'
driver.get(url)

In [41]:
def get_url(search_term):
    template = 'https://www.amazon.com/s?k={}&pd_rd_r=1dc02379-1097-4e2e-9b6f-9f458cdd5916&pd_rd_w=YOfcP&pd_rd_wg=wQEib&pf_rd_p=12129333-2117-4490-9c17-6d31baf0582a&pf_rd_r=9XFYQ5D3SSXQEDWWAMG9&ref=pd_gw_unk'
    search_term = search_term.replace(' ', '+')
    return template.format(search_term)

In [42]:
url = get_url('gaming chairs')
print(url)

https://www.amazon.com/s?k=gaming+chairs&pd_rd_r=1dc02379-1097-4e2e-9b6f-9f458cdd5916&pd_rd_w=YOfcP&pd_rd_wg=wQEib&pf_rd_p=12129333-2117-4490-9c17-6d31baf0582a&pf_rd_r=9XFYQ5D3SSXQEDWWAMG9&ref=pd_gw_unk


### Extract collection

In [43]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [44]:
# Danh sách các thẻ của sản phẩm(tên sp, giá bán, rating, ... )
results = soup.find_all('div', {'data-component-type':'s-search-result'})

In [45]:
len(results)

60

### Prototype with the record thứ 1

In [46]:
# Test với bản ghi đầu tiên
item = results[0]

In [1]:
atag = item.h2.a

NameError: name 'item' is not defined

In [62]:
description = atag.text.strip()

In [67]:
url = 'https://www.amazon.com' + atag.get('href')

In [71]:
price_parent = item.find('span', 'a-price')
price = price_parent.find('span', 'a-offscreen').text

'$188.12'

In [72]:
rating = item.i.text

'4.6 out of 5 stars'

In [74]:
review_count = item.find('span', {'class': 'a-size-base'}).text

'1,836'

### Generalize the pattern ( error handling)

In [82]:
#trích xuất dữ liệu - Extract in ETL
def extract_record(item):
    """Extract and return data from single record"""
    
    #description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    #price
    try:
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    #rating and amount of customer rating
    try:
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base'}).text
    except AttributeError:
        rating = ''
        review_count = ''
        
    result = (description, price, rating, review_count, url)
    
    return result

In [83]:
records = []
results = soup.find_all('div', {'data-component-type':'s-search-result'})

for item in results:
#   nhận giá trị result = {description, price, rating, review_count, url}
    record_temp = extract_record(item) 
    if record_temp:
        records.append(record_temp)

In [84]:
records[0]

('Amazon Basics Kids/Youth Gaming Recliner with Headrest and Back Pillow, 3+ Age Group, Black and Red',
 '$188.12',
 '4.6 out of 5 stars',
 '1,836',
 'https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A047620820RK7GCB0DFR3&url=%2FAmazonBasics-Gaming-Recliner-Headrest-Pillow%2Fdp%2FB07TJWWPJS%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dgaming%2Bchairs%26pd_rd_r%3D1dc02379-1097-4e2e-9b6f-9f458cdd5916%26pd_rd_w%3DYOfcP%26pd_rd_wg%3DwQEib%26pf_rd_p%3D12129333-2117-4490-9c17-6d31baf0582a%26pf_rd_r%3D9XFYQ5D3SSXQEDWWAMG9%26qid%3D1650122090%26sr%3D8-1-spons%26psc%3D1&qualifier=1650122090&id=3479174035583221&widgetName=sp_atf')

In [85]:
for row in records:
    print(row[1])

$188.12
$279.99
$99.99
$189.99
$99.99
$170.50
$99.99
$561.28
$99.00
$99.90
$115.34
$135.21
$272.64
$185.77
$99.99
$279.99
$176.91
$144.00
$226.10
$91.34
$129.72
$98.99
$504.00
$188.33
$373.99
$169.00
$183.69
$280.27
$226.10
$439.89
$303.29
$84.99
$419.99
$156.16
$142.45
$329.00
$179.99
$142.32
$160.01
$197.93
$173.13
$95.00
$142.17
$399.00
$209.98
$134.71
$352.99
$99.99
$99.00
$179.99
$156.16
$185.82


In [89]:
def get_url(search_term):
    template = 'https://www.amazon.com/s?k={}&qid=1650126684&ref=sr_pg_1'
    search_term = search_term.replace(' ', '+')
    # add term query in url
    url = template.format(search_term)
    
    #add page query placeholder
    url += '&page{}'
    
    return url

### Putting it all together

In [103]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=Service('C:\\Users\\CUONG NGUYEN\\.wdm\\drivers\\chromedriver\\win32\\100.0.4896.60\\chromedriver.exe'))

# get đường dẫn
def get_url(search_term):
    template = 'https://www.amazon.com/s?k={}&qid=1650126684&ref=sr_pg_1'
    search_term = search_term.replace(' ', '+')
    # add term query in url
    url = template.format(search_term)
    
    #add page query placeholder
    url += '&page{}'
    
    return url

#trích xuất dữ liệu - Extract in ETL
def extract_record(item):
    """Extract and return data from single record"""
    
    #description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    #price
    try:
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    #rating and amount of customer rating
    try:
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base'}).text
    except AttributeError:
        rating = ''
        review_count = ''
        
    result = (description, price, rating, review_count, url)
    
    return result

def main(search_term):
#     run amazon
#     url = 'https://www.amazon.com/'
#     driver.get(url)
    
#     run đường dẫn sản phẩm
    url = get_url(search_term)
    records = []
    
    for page in range(1, 11):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type':'s-search-result'})
        
        for item in results:
            record_temp = extract_record(item) 
            if record_temp:
                records.append(record_temp)
    driver.close()
    
    with open('result.csv', 'w', newline='', encoding='utf8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'ReviewCount', 'Url'])
        writer.writerows(records)

In [102]:
main('gaming chairs')